In [ ]:
#화면 캡쳐만 코드
import numpy as np
import cv2

cap = cv2.VideoCapture(0) # 노트북 웹캠을 카메라로 사용
cap.set(3,640) # 너비
cap.set(4,480) # 높이

ret, frame = cap.read() # 사진 촬영
frame = cv2.flip(frame, 1) # 좌우 대칭

cv2.imwrite('self camera test.jpg', frame) # 사진 저장
    
cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2

class FaceMosaic:
    def __init__(self, cascade_xml='haarcascades/haarcascade_frontalface_default.xml'):
        # Haar Cascade 얼굴 검출기 초기화
        self.face_cascade = cv2.CascadeClassifier(cascade_xml)
        
        # 웹캠 열기
        self.cap = cv2.VideoCapture(0)
        self.cap.set(3, 640)  # 너비
        self.cap.set(4, 480)  # 높이

    def detect_and_mosaic_faces(self, frame):
        # 프레임 좌우 반전
        frame = cv2.flip(frame, 1)  
        
        # 흑백으로 변환
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 얼굴 검출
        faces = self.face_cascade.detectMultiScale(gray, 1.05, 5)
        print("검출된 얼굴 수: " + str(len(faces)))

        if len(faces):
            for (x, y, w, h) in faces:
                # 얼굴 영역 잘라내기
                face_img = frame[y:y + h, x:x + w]
                
                # 얼굴 축소
                face_img = cv2.resize(face_img, dsize=(0, 0), fx=0.04, fy=0.04)
                
                # 얼굴 확대
                face_img = cv2.resize(face_img, (w, h), interpolation=cv2.INTER_AREA)
                
                # 검출된 얼굴 영역에 모자이크 처리
                frame[y:y + h, x:x + w] = face_img  

        return frame

    def run(self):
        while True:
            # 프레임 읽기
            ret, frame = self.cap.read()

            if not ret:
                print("프레임 캡처 실패")
                break

            # 얼굴 검출 및 모자이크 처리
            result_frame = self.detect_and_mosaic_faces(frame)
            
            # 결과 프레임 표시
            cv2.imshow('결과', result_frame)

            # 'Esc' 키를 누르면 종료
            k = cv2.waitKey(30) & 0xff
            if k == 27:  
                break

        # 웹캠 해제 및 창 닫기
        self.cap.release()
        cv2.destroyAllWindows()

def main():
    # FaceMosaic 클래스 인스턴스 생성 및 실행
    face_mosaic = FaceMosaic()
    face_mosaic.run()

if __name__ == "__main__":
    main()


검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 0
검출된 얼굴 수: 0
검출된 얼굴 수: 0
검출된 얼굴 수: 0
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1
검출된 얼굴 수: 1


날짜 시간도 표시

In [ ]:
import cv2
import datetime

class FaceMosaic:
    def __init__(self, scale_factor=0.04):
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.scale_factor = scale_factor

    def detect_faces(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
        return faces

    def mosaic_faces(self, frame, faces):
        for (x, y, w, h) in faces:
            face_img = frame[y:y + h, x:x + w]
            face_img = cv2.resize(face_img, dsize=(0, 0), fx=self.scale_factor, fy=self.scale_factor)
            face_img = cv2.resize(face_img, (w, h), interpolation=cv2.INTER_AREA)
            frame[y:y + h, x:x + w] = face_img
        return frame

def main():
    webcam = cv2.VideoCapture(0)
    webcam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    webcam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    face_mosaic = FaceMosaic()

    if not webcam.isOpened():
        print("웹캠을 열 수 없습니다")
        exit()

    while webcam.isOpened():
        ret, frame = webcam.read()
        if ret:
            faces = face_mosaic.detect_faces(frame)
            current_time = datetime.datetime.now().strftime("%H:%M:%S")

            if len(faces) > 0:
                print(f"{current_time}     X")
                frame = face_mosaic.mosaic_faces(frame, faces)
            else:
                print(f"{current_time}     0")

            cv2.imshow("얼굴 검출", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    webcam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
